In [1]:

from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.conversion.log import converter as log_converter

import pandas as pd 
import os



In [4]:
df = pd.read_pickle("../data/0_log_raw.pkl")
df

,org_group,res_country,org_country,org_res,org_involved,org_role,concept_name,impact,product,lifecycle_transition,timestamp,case_seq_num
0,V30,France,fr,Frederic,Org line A2,A2_4,Accepted,Medium,PROD582,In Progress,2010-03-31 16:59:42+00:00,1-364285768
1,V30,France,fr,Frederic,Org line A2,A2_4,Accepted,Medium,PROD582,In Progress,2010-03-31 17:00:56+00:00,1-364285768
2,V5 3rd,France,fr,Frederic,Org line A2,A2_5,Queued,Medium,PROD582,Awaiting Assignment,2010-03-31 17:45:48+00:00,1-364285768
3,V5 3rd,France,fr,Anne Claire,Org line A2,A2_5,Accepted,Medium,PROD582,In Progress,2010-04-06 16:44:07+00:00,1-364285768
4,V30,France,fr,Anne Claire,Org line A2,A2_4,Queued,Medium,PROD582,Awaiting Assignment,2010-04-06 16:44:38+00:00,1-364285768
...,...,...,...,...,...,...,...,...,...,...,...,...
65528,C9,Brazil,br,Lierson,Org line C,V3_2,Completed,Low,PROD424,In Call,2012-05-05 04:07:43+00:00,1-740866691
65529,C9,Brazil,br,Lierson,Org line C,V3_2,Accepted,Medium,PROD562,In Progress,2012-05-05 06:58:40+00:00,1-740866708
65530,C9,Brazil,br,Lierson,Org line C,V3_2,Accepted,Medium,PROD562,In Progress,2012-05-05 07:02:39+00:00,1-740866708
65531,C9,Brazil,br,Lierson,Org line C,V3_2,Completed,Medium,PROD562,In Call,2012-05-05 07:23:03+00:00,1-740866708


In [5]:

# filter out cases that does not end with Completed
df = df.sort_values(["case_seq_num", "timestamp"])
df = df.groupby("case_seq_num").filter(lambda trace: trace["concept_name"].iloc[-1] == "Completed").reset_index(drop=True)


In [6]:

# Filter out cases with very short or trivial sequences (>2 events)
df = df.groupby("case_seq_num").filter(lambda x: len(x) > 2)


In [7]:

# Filter out cases that they don't start with 'Accepted' or 'Queued'. 4 cases
df_sorted = df.sort_values(["case_seq_num", "timestamp"])
first_events = df_sorted.groupby("case_seq_num").first().reset_index()
valid_case_ids = first_events[first_events["concept_name"].isin(["Accepted", "Queued"])]["case_seq_num"]
df = df_sorted[df_sorted["case_seq_num"].isin(valid_case_ids)]


In [8]:

# Filter out outlier cases by duration
df.loc[:, "timestamp"] = pd.to_datetime(df["timestamp"])
df = df.sort_values(["case_seq_num", "timestamp"]) #sorting other time to make sure
case_durations = df.groupby("case_seq_num")["timestamp"].agg(["first", "last"]).reset_index()
case_durations["duration_days"] = (case_durations["last"] - case_durations["first"]).dt.total_seconds() / (24*60*60)
case_durations = case_durations.sort_values(["duration_days"])
duration_99th = case_durations["duration_days"].quantile(0.99)
valid_case_ids = case_durations[case_durations["duration_days"] < duration_99th ]["case_seq_num"]
df =df[df["case_seq_num"].isin(valid_case_ids)]


In [10]:

#filter out noise 
concept_name_proportions = df["concept_name"].value_counts(normalize=True)
valid_concepts = concept_name_proportions[concept_name_proportions >= 0.01].index
df = df[df["concept_name"].isin(valid_concepts)]


In [11]:

lctransitions_proportions = df["lifecycle_transition"].value_counts(normalize=True)
valid_lstransitions = lctransitions_proportions[lctransitions_proportions >= 0.01].index
df = df[df["lifecycle_transition"].isin(valid_lstransitions)]
